In [88]:
import pandas as pd
import re

In [89]:
status_df = pd.read_csv("fichier_csv/status.csv")
results_df = pd.read_csv("fichier_csv/results.csv")
constructors_df = pd.read_csv("fichier_csv/constructors.csv")

In [90]:
ecurie = ["McLaren","Mercedes","Ferrari","Red Bull"]
df = results_df.merge(constructors_df, on="constructorId").merge(status_df, on="statusId")
df = df[["name","status"]]
df = df[df["name"].isin(ecurie)]
df["status"] = df["status"].str.strip()
df

,name,status
0,McLaren,Finished
4,McLaren,Finished
7,Ferrari,Engine
12,Ferrari,Engine
13,Red Bull,Collision
...,...,...
26742,Mercedes,Finished
26743,Mercedes,Finished
26744,Red Bull,Finished
26748,McLaren,Finished


In [91]:
df.status.unique()

array(['Finished', 'Engine', 'Collision', 'Spun off', '+1 Lap',
       'Accident', '+2 Laps', 'Gearbox', 'Transmission', 'Hydraulics',
       'Brakes', 'Driveshaft', 'Suspension', 'Disqualified', 'Electrical',
       'Heat shield fire', 'Fuel pump', 'Track rod', 'Throttle', 'Tyre',
       'Puncture', 'Withdrew', 'Fuel pressure', 'Retired', 'Pneumatics',
       'Rear wing', 'Fuel system', '+4 Laps', 'Wheel rim', 'Exhaust',
       'Ignition', 'Stalled', 'Clutch', 'Steering', 'Oil pressure',
       'Wheel', '+5 Laps', 'Halfshaft', 'Broken wing', 'Differential',
       'Oil leak', 'Handling', 'Fuel leak', 'Wheel bearing', 'Injured',
       'Overheating', 'Water leak', '+3 Laps', 'Out of fuel',
       'Mechanical', 'Alternator', 'Injection', 'Battery', 'Radiator',
       'Electronics', 'Fire', '+6 Laps', 'Turbo', 'Water pump',
       'Distributor', 'Did not qualify', 'Injury', 'Not classified',
       '+9 Laps', 'Did not prequalify', '+7 Laps', '+8 Laps', 'Oil pump',
       '+30 Laps', '+14

In [92]:
df = df[~df["status"].isin([
    "Finished",
    '+1 Lap',
    '+2 Laps',
    '+4 Laps',
    '+5 Laps',
    '+3 Laps',
    '+6 Laps',
    'Did not qualify',
    '+9 Laps',
    'Did not prequalify',
    '+7 Laps',
    '+8 Laps',
    '+30 Laps', 
    '+14 Laps', 
    '+19 Laps', 
    '+12 Laps', 
    'Axle', 
    '+15 Laps',
    '+11 Laps',
    "Disqualified", 
    "Withdrew",
    "Retired",
    ])].reset_index(drop=True)

In [93]:
# Final categories
CATS = [
    "Power Unit",
    "Transmission",
    "Chassis/Brakes",
    "Electrical",
    "Hydraulics/Cooling",
    "On-track incident",
    "Admin/Regulatory",
]

# Exact matches (covers the canonical 69 labels from your dataset)
exact_map = {
    # Admin/Regulatory
    "Technical": "Technical",

    # On-track incident
    "Accident": "On-track incident",
    "Collision": "On-track incident",
    "Collision damage": "On-track incident",
    "Spun off": "On-track incident",
    "Damage": "On-track incident",

    # Electrical / hybrid
    "ERS": "Electrical",
    "Electrical": "Electrical",
    "Battery": "Electrical",
    "Alternator": "Electrical",
    "Electronics": "Electrical",
    "Magneto": "Electrical",

    # Power Unit
    "Engine": "Engine/Fuel/Exhaust",
    "Power Unit": "Engine/Fuel/Exhaust",
    "Turbo": "Engine/Fuel/Exhaust",
    "Ignition": "Engine/Fuel/Exhaust",
    "Spark plugs": "Engine/Fuel/Exhaust",
    "Distributor": "Engine/Fuel/Exhaust",
    "Power loss": "Engine/Fuel/Exhaust",
    "Overheating": "Engine/Fuel/Exhaust",
    "Out of fuel": "Engine/Fuel/Exhaust",
    "Oil leak": "Engine/Fuel/Exhaust",
    "Oil pressure": "Engine/Fuel/Exhaust",
    "Oil pump": "Engine/Fuel/Exhaust",
    "Pneumatics": "Engine/Fuel/Exhaust",
    "Throttle": "Engine/Fuel/Exhaust",
    "Fuel pump": "Engine/Fuel/Exhaust",
    "Fuel pressure": "Engine/Fuel/Exhaust",
    "Fuel system": "Engine/Fuel/Exhaust",
    "Fuel leak": "Engine/Fuel/Exhaust",
    "Injection": "Engine/Fuel/Exhaust",
    "Exhaust": "Engine/Fuel/Exhaust",
    "Heat shield fire": "Engine/Fuel/Exhaust",
    "Fire": "Engine/Fuel/Exhaust",
    "Stalled": "Engine/Fuel/Exhaust",

    # Transmission / drivetrain
    "Transmission": "Transmission",
    "Gearbox": "Transmission",
    "Clutch": "Transmission",
    "Differential": "Transmission",
    "Driveshaft": "Transmission",
    "Halfshaft": "Transmission",

    # Chassis/Brakes (incl. tyres, aero bits)
    "Brakes": "Chassis/Handling",
    "Brake duct": "Chassis/Handling",
    "Suspension": "Chassis/Handling",
    "Steering": "Chassis/Handling",
    "Track rod": "Chassis/Handling",
    "Tyre": "Chassis/Handling",
    "Puncture": "Chassis/Handling",
    "Wheel": "Chassis/Handling",
    "Wheel nut": "Chassis/Handling",
    "Wheel rim": "Chassis/Handling",
    "Wheel bearing": "Chassis/Handling",
    "Front wing": "Chassis/Handling",
    "Rear wing": "Chassis/Handling",
    "Broken wing": "Chassis/Handling",
    "Wing": "Chassis/Handling",
    "Undertray": "Chassis/Handling",
    "Floor": "Chassis/Handling",
    "Vibrations": "Chassis/Handling",
    "Mechanical": "Chassis/Handling",
    "Handling": "Chassis/Handling",
    
    # Injuries
    "Injured": "Injuries",
    "Injury": "Injuries",

    # Hydraulics/Cooling
    "Hydraulics": "Hydraulics/Cooling",
    "Radiator": "Hydraulics/Cooling",
    "Water leak": "Hydraulics/Cooling",
    "Water pressure": "Hydraulics/Cooling",
    "Water pump": "Hydraulics/Cooling",
    "Cooling": "Hydraulics/Cooling",
    
}


In [94]:
def map_status(s: str) -> str:
    s = s.strip()
    if s in exact_map:
        return exact_map[s]
    """low = s.lower()
    for cat, pat in regex_rules:
        if re.search(pat, low):
            return cat"""
    # Last resort (should not trigger if your list is complete)
    return "Other/Unknown"

df["category"] = df["status"].apply(map_status)

In [95]:
df[df["category"] == "Other/Unknown"].status.unique()

array(['Not classified'], dtype=object)

In [97]:
for marque in ecurie:
    df_une_marque = df[df["name"] == marque][["name","category"]]
    res = df_une_marque.groupby("category").count()
    print(marque)
    res = res.sort_values("name",ascending=False)
    print(res.index.values.tolist())
    print(res.name.values.tolist())

McLaren
['Engine/Fuel/Exhaust', 'On-track incident', 'Transmission', 'Chassis/Handling', 'Electrical', 'Hydraulics/Cooling', 'Other/Unknown', 'Injuries']
[195, 188, 62, 53, 33, 12, 3, 1]
Mercedes
['On-track incident', 'Engine/Fuel/Exhaust', 'Chassis/Handling', 'Transmission', 'Hydraulics/Cooling', 'Electrical']
[29, 22, 11, 5, 3, 2]
Ferrari
['Engine/Fuel/Exhaust', 'On-track incident', 'Transmission', 'Chassis/Handling', 'Electrical', 'Hydraulics/Cooling', 'Other/Unknown', 'Injuries']
[270, 203, 100, 74, 36, 13, 4, 2]
Red Bull
['On-track incident', 'Engine/Fuel/Exhaust', 'Transmission', 'Chassis/Handling', 'Electrical', 'Hydraulics/Cooling', 'Technical']
[54, 24, 17, 16, 11, 10, 1]
